In [31]:
'''from Training.generator_memory import generator
import numpy as np
from SSM.hippo import hippo_legs, discretize_bilinear
from SSM.helpers import sigmoid, binary_cross_entropy
from SSM.model import SimpleSSM
'''



from generator_memory import generator 
import numpy as np
from hippo import hippo_legs, discretize_bilinear
from helpers import sigmoid, binary_cross_entropy
from mmodel import SimpleSSM

In [93]:
#data parameters


#set data parameters
n_sequence = 1
n_train = 10000
n_test = 1000
time_step = 10
#n_batch

gen = generator(n_sequence, time_step, n_train)

x_train, y_train = gen.seq()

print(x_train[0:10])

gen = generator(n_sequence, time_step, n_test)

x_test, y_test = gen.seq()



[[8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [6. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [7. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [7. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [7. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]]


In [15]:
xx = x_train[0].reshape(-1, 1)
xx.shape
yy = y_train[0].reshape(-1,1)
yy.shape

(12, 1)

In [95]:
model = SimpleSSM(input_dim = 1,
        output_dim = 10)
#print(y_train[0])
print(model.A_ct)
print(model.A)
prev = 1000000000
prev_a = 5
for j in range(100):
    
    epoch_loss = 0.0 
    for i in range(x_train.shape[0]):
        loss, grads = model.loss_and_grads_softmax(x_train[i].reshape(-1, 1), y_train[i].reshape(-1,1))
        model.step(grads, lr = 1e-5)
        epoch_loss += loss
    if epoch_loss/x_train.shape[0] > prev and prev_a != 5:
        prev_a = model.A.copy()
    if j % 10 == 0:
        if epoch_loss/x_train.shape[0] < prev:
            prev = epoch_loss/x_train.shape[0]
        else:
            break
            
    print('average loss= ' + str(epoch_loss/x_train.shape[0]))

print(prev_a)
print(model.A)



[[ -1.           0.           0.           0.           0.
    0.           0.           0.        ]
 [ -1.73205081  -2.           0.           0.           0.
    0.           0.           0.        ]
 [ -2.23606798  -3.87298335  -3.           0.           0.
    0.           0.           0.        ]
 [ -2.64575131  -4.58257569  -5.91607978  -4.           0.
    0.           0.           0.        ]
 [ -3.          -5.19615242  -6.70820393  -7.93725393  -5.
    0.           0.           0.        ]
 [ -3.31662479  -5.74456265  -7.41619849  -8.77496439  -9.94987437
   -6.           0.           0.        ]
 [ -3.60555128  -6.244998    -8.06225775  -9.53939201 -10.81665383
  -11.95826074  -7.           0.        ]
 [ -3.87298335  -6.70820393  -8.66025404 -10.24695077 -11.61895004
  -12.84523258 -13.96424004  -8.        ]]
[[ 3.33333333e-01 -3.37864825e-16 -2.06854932e-16  1.93156033e-16
   1.87791344e-16 -6.85528257e-17 -3.86372710e-16 -2.22044605e-16]
 [-5.77350269e-01  3.90787006e-16 

In [96]:
#once the loss starts to increase, it increaes indefintelyt

x_train[1], y_train[1], model.forward(x_train[1].reshape(-1, 1))

(array([3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 9., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.]),
 (array([[   7.55173353,    5.59149507,    5.82344089,    5.95003953,
             6.37278799,    6.25761238,    5.99366157,    6.15164421,
             5.3566511 ,    4.90989457],
         [  -6.67909042,   -9.7242324 ,   -9.33534108,   -9.5595092 ,
            -9.60224783,   -9.48047518,   -9.21850173,   -9.88078375,
            -9.81388211,   -9.71344097],
         [  17.31314618,   15.03409328,   15.04258247,   14.74227444,
            14.56394812,   14.62013913,   14.80872308,   14.69307592,
            13.84614234,   13.68682623],
         [ -16.9832152 ,  -22.43258531,  -22.4160264 ,  -22.46371932,
           -23.01071148,  -23.12015259,  -22.27293978,  -23.38786134,
           -22.12688958,  -21.62975348],
         [  37.90772529,   35.25580916,   35.46853831,   34.31389983,
            33.98524419,   35.21468172,   34.31016196,   34.35048414,
            33.29558

In [97]:
for i in range(10):
    #print(x_train[i])
    #print(y_train[i])
    logits, _ = model.forward(x_train[i].reshape(-1,1))
    
    pred_digits = np.argmax(logits, axis=1)
    #print(pred_digits)
    if y_train[i].all() == pred_digits.all():
        print(pred_digits, y_train[i])

[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 8.]
[0 0 0 0 0 0 0 0 0 0 0 5] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 6.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 7.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 7.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 7.]


In [58]:
x_test.shape

(1000, 3)

In [ ]:
N = x_test.shape[0]
total_loss = 0.0
total_correct = 0
total_bits = 0

for i in range(N):
    u = x_test[i].reshape(-1, 1)      # (T, 1)
    tgt = y_test[i].reshape(-1,1)    # (T, 1)

    # forward pass only (no grads)
    logits, _ = model.forward(u)

    # loss for this sequence
    pred_digits = np.argmax(logits, axis=1)


    # compute accuracy for this sequence
    #print(tgt.reshape(-1), pred_digits)
    if tgt.reshape(-1)[-1] == pred_digits[-1]:
        total_correct += 1
    
    


accuracy = total_correct / N

print(accuracy)
